In [15]:
import warnings
warnings.filterwarnings('ignore')
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew

In [16]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
os.environ["OPENAI_MODEL_NAME"]='gpt-4-turbo'

In [17]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [18]:
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal = "Identify and book an appropriate venue, "
    "based on event requirements.",
    tools = [search_tool, scrape_tool],
    verbose=True,
    backstory = (
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."    
    )
)

In [19]:
logistics_manager = Agent(
    role = "Logistics Manager",
    goal = "Manage all logistics foir the event, "
    "including catering and equipment.",
    tools = [search_tool, scrape_tool],
    verbose = True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."    
    )

)

In [20]:
marketing_comms_expert = Agent(
    role= "Marketing and Communications Agent",
    goal = "Effecively market the event and " 
    "communicate with participants.",
    tools = [search_tool, scrape_tool],
    verbose = True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

In [22]:
from pydantic import BaseModel
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [30]:
venue_task = Task(
    description="Find a venue in {city} that meets" 
        "the criteria for {event_topic}",
    expected_output="All the details of a " 
        "specifically chosen venue you found to "
        "accomodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
    agent = venue_coordinator
)

In [31]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                "including catering and equipment setup.",
    human_input=True,
    #async_execution=True,
    agent=logistics_manager
)

In [32]:
marketing_task = Task(
    description= "Promote the {event_topic} " 
        "aiming to engage at least {expected_participants} potential attendees" ,
    expected_output="Report on Marketing activities and attendee "
        "engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",
    agent = marketing_comms_expert
)

In [ ]:
event_management_crew = Crew(
    agents=[venue_coordinator, marketing_comms_expert, logistics_manager],

    tasks=[venue_task, logistics_task, marketing_task],

    verbose=True

)

In [ ]:
event_details = {
    "event_topic": "Tech Innovation Conference",
    "event_description"
}